# Heart Statlog dataset with LionForests V2

In [3]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-18])
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\')
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\')

In [4]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score

from anchor import anchor_tabular


Firstly, we load the dataset and we set the feature and class names

In [9]:
heart = Dataset()
X, y, feature_names, class_names = heart.load_heart() #Feature names should not contain any space
y = np.array(y)
one_h_percent = int(min(10*len(X)/100,100))
print("Instances:",one_h_percent)
new_fn = []
for i in feature_names:
    new_fn.append(i.replace(' ','_'))
feature_names = new_fn

Instances: 27


Then we initiate LionForests

In [11]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names):
    parameters = [{
        'max_depth': [5],
        'max_features': [0.75],
        'bootstrap': [False],
        'min_samples_leaf' : [5],
        'n_estimators': [500]
    }]
    scaler = MinMaxScaler(feature_range=(-1,1))
    lf = LionForests(None, False, scaler, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            return rule, cover, def_predictions

        #Anchors =======================================================================================
        explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
        temp_frame = pd.DataFrame(np.hstack((train,y_train.reshape(len(y_train),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)
        temp_frame = temp_frame.replace({"class": {1: 2}})
        temp_frame = temp_frame.replace({"class": {0: 1}})
        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'heart', random_state=123)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)

        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=123,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification')
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        covered = False
                    if instance[k] <= rule[feature][1][1]:
                        covered = False
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        covered = False
                else:
                     if instance[k] > rule[feature][0][1]: #<=
                        covered = False
        if covered:
            return 1
        else:
            return 0

    gt = GlobalSurrogateTree(train,lf.model.predict(train),feature_names,'c')
    lt = LocalSurrogateTree(train,lf.model.predict(train),feature_names,'c',150)

    loo = LeaveOneOut()
    loo.get_n_splits(test)
    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    for train_index, test_index in loo.split(test):
        x_train_temp, x_test_temp = test[train_index], test[test_index]
        y_train_temp, y_test_temp = test_predictions[train_index], test_predictions[test_index]
        x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
        x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op = method(test_index[0])
                coverage = 0
                precision = []
                co = 0
                for i in x_train_temp_lf:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions = method(np.concatenate((x_test_temp,x_train_temp)))
                f_precision[name].append(precision_score(predictions,y_train_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                rule = method(x_test_temp_lf[0])
                coverage = 0
                precision = []
                co = 0
                for i in x_train_temp_lf:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            else:
                rule, prediction = method(x_test_temp[0])
                coverage = 0
                precision = []
                co = 0 
                for i in x_train_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision

In [12]:
#27 instances
total_results = []
rs = [7, 77, 777, 7777, 77777, 5, 55, 555, 5555, 55555]
for random_s in rs:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=one_h_percent, stratify=y, random_state=random_s)
    results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
    total_results.append(results)

len self.path_detail
27
Working on CHIRPS for instance 0 of 27
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 80 patterns from 426 trees for batch_idx 0
start score sort for batch_idx 0 (80) patterns
start merge rule for batch_idx 0 (80) patterns
[('chest_pain', True, 0.66667), ('number_of_major_vessels', True, -0.66667)]
0.9361702127659575 0.35946009945536345 0.1709849812361538 0.2689971656093942
merge complete for batch_idx 0 (80) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
reduced 8 patterns out of 74 by numeric redundancy
found 66 patterns from 452 trees for batch_idx 1
start score sort for batch_idx 1 (66) patterns
start merge rule for batch_idx 1 (66) patterns
[('chest_pain', True, 0.66667), ('number_of_major_vessels', True, -0.66667)]
0.9361702127659575 0.35946009945536345 0.15629264227015172 0.22343175942305085
merge complete for batch_idx 1 (66) patterns
start get explai

[('chest_pain', False, 0.5416666716337204), ('number_of_major_vessels', False, -0.66667)]
0.9538461538461539 0.25426829268292683 0.18602634825933173 0.2669220945083013
merge complete for batch_idx 16 (31) patterns
start get explainer for batch_idx 16
as_chirps for batch_idx 17
start mining for batch_idx 17 with support = 0.1
found 48 patterns from 432 trees for batch_idx 17
start score sort for batch_idx 17 (48) patterns
start merge rule for batch_idx 17 (48) patterns
[('chest_pain', True, 0.66667), ('number_of_major_vessels', True, -0.66667)]
0.9361702127659575 0.35946009945536345 0.09374773977101254 0.12263374485596713
merge complete for batch_idx 17 (48) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
found 58 patterns from 491 trees for batch_idx 18
start score sort for batch_idx 18 (58) patterns
start merge rule for batch_idx 18 (58) patterns
[('number_of_major_vessels', True, -0.66667), ('resting_blood_pres

[('maximum_heart_rate_achieved', True, 0.17061), ('reversable_defect', False, -0.375)]
0.9215686274509803 0.19632099198689829 0.36576808482418116 0.5044378698224846
merge complete for batch_idx 5 (35) patterns
start get explainer for batch_idx 5
as_chirps for batch_idx 6
start mining for batch_idx 6 with support = 0.1
found 43 patterns from 450 trees for batch_idx 6
start score sort for batch_idx 6 (43) patterns
start merge rule for batch_idx 6 (43) patterns
[('chest_pain', False, 0.5416666716337204), ('number_of_major_vessels', False, -0.7916666716337204)]
0.9508196721311475 0.2381119494648184 0.14971882728600197 0.23173536175482048
merge complete for batch_idx 6 (43) patterns
start get explainer for batch_idx 6
as_chirps for batch_idx 7
start mining for batch_idx 7 with support = 0.1
reduced 3 patterns out of 34 by numeric redundancy
found 31 patterns from 500 trees for batch_idx 7
start score sort for batch_idx 7 (31) patterns
start merge rule for batch_idx 7 (31) patterns
[('chest_

[('number_of_major_vessels', True, -0.66667), ('resting_blood_pressure', True, -0.03774), ('reversable_defect', True, -0.25)]
0.9642857142857143 0.3301282051282051 0.4929026692416416 0.551117047110248
merge complete for batch_idx 21 (63) patterns
start get explainer for batch_idx 21
as_chirps for batch_idx 22
start mining for batch_idx 22 with support = 0.1
found 61 patterns from 493 trees for batch_idx 22
start score sort for batch_idx 22 (61) patterns
start merge rule for batch_idx 22 (61) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25), ('oldpeak', True, -0.29032)]
0.9772727272727273 0.34959349593495936 0.15692599202514818 0.1857646540326134
merge complete for batch_idx 22 (61) patterns
start get explainer for batch_idx 22
as_chirps for batch_idx 23
start mining for batch_idx 23 with support = 0.1
found 77 patterns from 375 trees for batch_idx 23
start score sort for batch_idx 23 (77) patterns
start merge rule for batch_idx 23 (77) patterns
[('chest_pain'

[('number_of_major_vessels', False, -0.66667), ('oldpeak', False, -0.8537), ('reversable_defect', False, -0.375)]
0.94 0.193757462050145 0.3421770235142633 0.43016759776536323
merge complete for batch_idx 11 (30) patterns
start get explainer for batch_idx 11
as_chirps for batch_idx 12
start mining for batch_idx 12 with support = 0.1
found 53 patterns from 495 trees for batch_idx 12
start score sort for batch_idx 12 (53) patterns
start merge rule for batch_idx 12 (53) patterns
[('chest_pain', True, 0.66667), ('oldpeak', True, -0.37143)]
0.907563025210084 0.4328048780487805 0.17104554475645833 0.22549753104893008
merge complete for batch_idx 12 (53) patterns
start get explainer for batch_idx 12
as_chirps for batch_idx 13
start mining for batch_idx 13 with support = 0.1
found 52 patterns from 479 trees for batch_idx 13
start score sort for batch_idx 13 (52) patterns
start merge rule for batch_idx 13 (52) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25)]
0.946808

...

[('chest_pain', True, 0.66667), ('oldpeak', True, -0.18367), ('reversable_defect', True, -0.25)]
0.9782608695652174 0.36585365853658536 0.1380541563138069 0.23841961852861038
merge complete for batch_idx 22 (74) patterns
start get explainer for batch_idx 22
as_chirps for batch_idx 23
start mining for batch_idx 23 with support = 0.1
found 63 patterns from 398 trees for batch_idx 23
start score sort for batch_idx 23 (63) patterns
start merge rule for batch_idx 23 (63) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25), ('oldpeak', True, -0.20124)]
0.9782608695652174 0.36585365853658536 0.18515672658602444 0.25762756443976853
merge complete for batch_idx 23 (63) patterns
start get explainer for batch_idx 23
as_chirps for batch_idx 24
start mining for batch_idx 24 with support = 0.1
found 26 patterns from 200 trees for batch_idx 24
start score sort for batch_idx 24 (26) patterns
start merge rule for batch_idx 24 (26) patterns
[('maximum_heart_rate_achieved', True, 

found 61 patterns from 500 trees for batch_idx 13
start score sort for batch_idx 13 (61) patterns
start merge rule for batch_idx 13 (61) patterns
[('number_of_major_vessels', True, -0.66667), ('resting_blood_pressure', True, -0.03774), ('reversable_defect', True, -0.25)]
0.9647058823529412 0.3341541588492808 0.4322404757510354 0.546832332058984
merge complete for batch_idx 13 (61) patterns
start get explainer for batch_idx 13
as_chirps for batch_idx 14
start mining for batch_idx 14 with support = 0.1
reduced 1 patterns out of 46 by numeric redundancy
found 45 patterns from 470 trees for batch_idx 14
start score sort for batch_idx 14 (45) patterns
start merge rule for batch_idx 14 (45) patterns
[('chest_pain', False, 0.5416666716337204), ('reversable_defect', False, -0.25)]
0.8888888888888888 0.27226998888693926 0.09134854220010638 0.12197028928850664
merge complete for batch_idx 14 (45) patterns
start get explainer for batch_idx 14
Working on CHIRPS for instance 15 of 27
as_chirps for 

[('chest_pain', False, 0.5416666716337204), ('reversable_defect', False, 0.375)]
0.8611111111111112 0.2686338052191711 0.6368964016672565 0.6144914073385974
merge complete for batch_idx 3 (26) patterns
start get explainer for batch_idx 3
as_chirps for batch_idx 4
start mining for batch_idx 4 with support = 0.1
reduced 2 patterns out of 37 by numeric redundancy
found 35 patterns from 500 trees for batch_idx 4
start score sort for batch_idx 4 (35) patterns
start merge rule for batch_idx 4 (35) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25)]
0.967391304347826 0.3621951219512195 0.15377618186706807 0.23703156728816693
merge complete for batch_idx 4 (35) patterns
start get explainer for batch_idx 4
Working on CHIRPS for instance 5 of 27
as_chirps for batch_idx 5
start mining for batch_idx 5 with support = 0.1
found 32 patterns from 243 trees for batch_idx 5
start score sort for batch_idx 5 (32) patterns
start merge rule for batch_idx 5 (32) patterns
[('chest_pai

[('number_of_major_vessels', True, -0.66667), ('reversable_defect', True, 0.75)]
0.944954128440367 0.4175609756097561 0.11857750315655496 0.1447278679579868
merge complete for batch_idx 20 (76) patterns
start get explainer for batch_idx 20
as_chirps for batch_idx 21
start mining for batch_idx 21 with support = 0.1
found 62 patterns from 490 trees for batch_idx 21
start score sort for batch_idx 21 (62) patterns
start merge rule for batch_idx 21 (62) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25)]
0.967391304347826 0.3621951219512195 0.09476207812836461 0.15155502392344505
merge complete for batch_idx 21 (62) patterns
start get explainer for batch_idx 21
as_chirps for batch_idx 22
start mining for batch_idx 22 with support = 0.1
found 36 patterns from 492 trees for batch_idx 22
start score sort for batch_idx 22 (36) patterns
start merge rule for batch_idx 22 (36) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25)]
0.967391304347826 0.

[('number_of_major_vessels', False, -0.7916666716337204), ('reversable_defect', False, -0.25)]
0.9137931034482759 0.2227291337661578 0.27211626372754383 0.3929893315915523
merge complete for batch_idx 11 (39) patterns
start get explainer for batch_idx 11
as_chirps for batch_idx 12
start mining for batch_idx 12 with support = 0.1
found 62 patterns from 495 trees for batch_idx 12
start score sort for batch_idx 12 (62) patterns
start merge rule for batch_idx 12 (62) patterns
[('number_of_major_vessels', True, -0.66667), ('resting_blood_pressure', True, -0.03645), ('reversable_defect', True, -0.25)]
0.9523809523809523 0.32692307692307687 0.4136769635331231 0.5322746654903692
merge complete for batch_idx 12 (62) patterns
start get explainer for batch_idx 12
as_chirps for batch_idx 13
start mining for batch_idx 13 with support = 0.1
found 65 patterns from 463 trees for batch_idx 13
start score sort for batch_idx 13 (65) patterns
start merge rule for batch_idx 13 (65) patterns
[('number_of_ma

In [45]:
#Me 27
full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/one_h_percent)
        rule_length[name].append(i[2][name]/one_h_percent)
        f_precision[name].append(np.array(i[3][name]).mean())
for name, method in i[0].items():
    print(name,np.array(full_coverage[name]).mean(),',',np.array(full_coverage[name]).std(),
          np.array(rule_length[name]).mean(),',',np.array(rule_length[name]).std(),
          np.array(f_precision[name]).mean(),',',np.array(f_precision[name]).std())

gs 0.15584045584045583 , 0.04179497860806636 3.525925925925926 , 0.4189607627371635 0.980667578035999 , 0.031888976464620286
ls 0.16324786324786325 , 0.02500963686308232 3.6111111111111116 , 0.22725776232062891 0.9402740134082368 , 0.039845711604130654
an 0.1965811965811966 , 0.05023059463188496 2.537037037037037 , 0.2959488703847849 0.5737330047167004 , 0.11430024986599777
lf 0.004558404558404559 , 0.0052455705525703625 7.681481481481481 , 0.6115486649051938 nan , nan
df 1.0 , 0.0 0.6518518518518518 , 1.021201852521804 0.8925925925925926 , 0.05844345865947964
ch 0.42592592592592593 , 0.08323038908979345 2.4703703703703708 , 0.24903104405947019 0.4269230769230769 , 0.08505132456729224
